In [23]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config("Users/odl_user_281290/config.json")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

StatementMeta(b6712890-fba9-472d-a032-aa60f0c8e41e, 1, 28, Finished, Available, Finished)

Workspace name: quick-starts-ws-281290
Azure region: westeurope
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-281290


In [8]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "project-1-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# try:
#    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
#    print("Found existing compute target: ", cluster_name)
#except ComputeTargetException:
print("Creating a new compute target: ", cluster_name)
compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)

StatementMeta(b6712890-fba9-472d-a032-aa60f0c8e41e, 1, 13, Finished, Available, Finished)

Creating a new compute target:  project-1-cluster
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [26]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--c': uniform(0.001, 10.0),
    '--max_iter': choice(50, 100, 150)
}) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(
    slack_factor=0.1,
    evaluation_interval=1,
    delay_evaluation=5
) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='Users/odl_user_281290/conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory='Users/odl_user_281290/',
    script='train.py',
    arguments=['--c', 1.0, '--max_iter', 100],
    compute_target=compute_target,
    environment=sklearn_env
) ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=1
) ### YOUR CODE HERE ###

StatementMeta(b6712890-fba9-472d-a032-aa60f0c8e41e, 1, 31, Finished, Available, Finished)

In [27]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hd_run = exp.submit(config=hyperdrive_config)
RunDetails(hd_run).show()
hd_run.wait_for_completion(show_output=True)

StatementMeta(b6712890-fba9-472d-a032-aa60f0c8e41e, 1, 32, Submitted, Running, Running)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_d5f6aa33-5dfa-448b-a8d9-3fcda4763d07
Web View: https://ml.azure.com/runs/HD_d5f6aa33-5dfa-448b-a8d9-3fcda4763d07?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-281290/workspaces/quick-starts-ws-281290&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-05-24T09:58:25.9544720Z][GENERATOR][DEBUG]Sampled 1 jobs from search space 
[2025-05-24T09:58:26.4326043Z][SCHEDULER][INFO]Scheduling job, id='HD_d5f6aa33-5dfa-448b-a8d9-3fcda4763d07_0' 
[2025-05-24T09:58:27.8501286Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_d5f6aa33-5dfa-448b-a8d9-3fcda4763d07_0' 


In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hd_run.get_best_run_by_primary_metric()
model = best_run.register_model(model_name='best_model', model_path='outputs/')
joblib.dump(model, 'best_model.joblib')
print('Best run ID: ', best_run.id)

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=url_path)

In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)
RunDetails(automl_run).show()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run, fitted_model = automl_run.get_output()
joblib.dumb(fitted_model, 'best_automl_model.pkl')
print('Best model saved successfully!')